# La Possession

---

> L'ownership ou la possession en français est un concept du langage de programmation Rust qui définit à qui appartient telle ou telle variable, ou dans quel bloc elle existe ou n'existe plus.

## Le transfert de possession et la copie

### Transfert de possession ou move

- Voici un exemple illustrant le transfert de possession à une fonction :

In [30]:
#[derive(Debug, Default)] // La structure n'est pas Copy !
struct Point {
    i: usize,
    j: usize,
}

fn debug_point(p: Point) {
    dbg!(p);
}

fn exemple() {
    let p: Point = Point::default();
    dbg!(p.i);
    dbg!(p.j);
    // Ici la variable p appartient toujours au bloc ...
    debug_point(p); // Transfert de l'ownership de la variable p a la fonction debug_point.
    // dbg!(p.i); // La variable p n'existe plus ici ! 
}
exemple()

[src/lib.rs:10] p.i = 0
[src/lib.rs:11] p.j = 0
[src/lib.rs:42] p = Point {
    i: 0,
    j: 0,
}


()

> Pour que la variable de type X soit transférée (ou moved) à la fonction, le type de doit pas implémenter le trait Copy. L'implémentation du trait Copy aurait pour effet que seulement une copie de la variable est passée a la fonction tandis que le bloc appelant garderait la possession de l'original.

### Les types implicitement copiés

- Tous les types numériques présentés au début de ce cours implémentent le trait Copy (dont usize donc) :

In [16]:
fn debug_usize(i: usize) {
    dbg!(i);
}

fn exemple() {
    let i: usize = 42;
    dbg!(i);
    // Ici la variable i appartient toujours au bloc ...
    debug_usize(i); // Une copie de i est passee a la fonction debug_usize
    dbg!(i); // La variable i existe toujours ici !
}
exemple()

[src/lib.rs:10] i = 42
[src/lib.rs:32] i = 42
[src/lib.rs:13] i = 42


()

> Si j'implémentais le trait Copy pour une structure ou une énumération (via Derive par exemple), elle serait aussi implicitement copiée.

**NB : POUR LES TYPES QUI NE SONT PAS COPY, LE TRANSFERT DE POSSESSION NE PEUT SE FAIRE QUE VIA LES FONCTIONS, LES MÉTHODES ET LES CLOSURES (DANS LE CAS D'UTILISATION DU MOT-CLEF MOVE)**

## Reprendre la possession

> Il existe deux moyens de garder ou de reprendre la possession d'une variable qui a été transférée :
> - L'emprunt d'une référence (borrowing).
> - Le retour de la fonction.

- Nous traiterons longuement de la première solution, qui fonctionne avec les références, un peu plus tard dans ce cours, alors voyons la seconde solution :

In [23]:
#[derive(Debug, Default)] // La structure n'est pas Copy !
struct Point {
    i: usize,
    j: usize,
}

fn debug_point(p: Point) -> Point {
    dbg!(p.i);
    Point {
        i: p.i,
        j: p.j,
    }
}

fn exemple() {
    let p: Point = Point::default();
    dbg!(p.i);
    // Ici la variable p appartient toujours au bloc ...
    let p = debug_point(p); // Ce n'est plus tout a fait le meme p, mais la copie de la copie.
    dbg!(p.i); // On peut acceder a ce nouveau p
}
exemple()

[src/lib.rs:10] p.i = 0
[src/lib.rs:36] p.i = 0
[src/lib.rs:13] p.i = 0


()

> Cette solution n'est pas l'idéal en termes d'utilisation de la pile, mais elle a ses adeptes, et au moins, elle a le mérite de fonctionner. Seulement, on aurait aimé avoir la garantie que la variable retournée contienne des données identiques à l'original, or, cette solution ne peut la garantir puisque la variable de type Point retournée est nouvelle ! On aurait très bien pu écrire cela dans la fonction :
```
fn debug_point(p: Point) -> Point {
    dbg!(p.i);
    Point {
        i: p.i + 3,
        j: p.j + 333,
    }
}
```
> Et c'est un peu pour cette raison que le passage par référence (borrowing) est une bien meilleure solution (enfin, a mon gout).

## La fausse modification

> Un autre problème peut se poser, mais cette fois-ci pour les types qui implémentent le trait Copy. Puisque la valeur n'est que copiée, toutes les modifications locales dans la fonction appelée n'auront aucun impact sur l'original.

- En voici une illustration :

In [25]:
fn increment_usize(mut i: usize) {
    i += 1;
}

fn exemple() {
    let i: usize = 42;
    dbg!(i);
    // Ici la variable i appartient toujours au bloc ...
    increment_usize(i); // Une copie de i est passee a la fonction debug_usize
    dbg!(i); // La variable i existe toujours ici et n'a pas change !
}
exemple()

[src/lib.rs:10] i = 42
[src/lib.rs:13] i = 42


()

- Sans utiliser de références mutables, la seule solution à ce problème consiste à se servir de ce qui a été vue dans la partie précédente, la valeur de retour de la fonction :

In [27]:
fn increment_usize(i: usize) -> usize {
    i + 1
}

fn exemple() {
    let i: usize = 42;
    dbg!(i);
    // Ici la variable i appartient toujours au bloc ...
    let i = increment_usize(i); // Nouveau i
    dbg!(i); // La variable a changee ce coup-ci
}
exemple()

[src/lib.rs:10] i = 42
[src/lib.rs:13] i = 43


()

## Conclusions

> Nous avons vu dans ce chapitre les bases de l'ownership et différentes difficultés que vous pouvez rencontrer. Sachez que tous les problèmes évoqués dans cette partie trouveront une solution satisfaisante avec les références et l'emprunt.

## Exercices

### Je viens de définir une fonction drop() pour un type alloue dynamiquement qui n'implémente donc pas le trait Copy, quelle est la conséquence de l'appel à cette fonction ?

```
fn drop(s: String) {}
```

> Si j'utilise cette fonction et que donc je lui envoie une String, je la perdrai définitivement ! Comme elle est allouée dynamiquement, j'imagine que je restituerai la mémoire utilisée par cette String au système.